# Exercise 3: Modelling transient fluid flow in the crust, background information

Elco Luijendijk

October 2019

<elco.luijendijk@geo.uni-goettingen.de>


## Introduction

In this exercise we will move from simulating steady-state diffusion to transient (time-dependent) diffusion of fluids in the crust. We will first construct a numerical model of transient fluid flow and then apply this model to a real world case study. You will have the choice between two case studies, one in which we compare predictions of our 1D model to data on seismicity induced by changes in hydraulic head and pore pressure at depth and a second case study where we will study the resilience of a spring to a change in precipitation and recharge. 


## Objectives

* Learn to simulate transient fluid, heat or solute transport in the crust
* Learn to combine numerical models and geoscience datasets to study geological processes.
* Learn how fluid transport behaves over long(ish) timescales

Do not hesitate to ask questions if you get stuck anywhere. You can reach me by email at <eluijen@gwdg.de> or pass by at my office, room 122 in the Structural Geology dept.


**Deadline: 21 february 2020**

*Good luck !*

## The transient fluid, heat and solute transport equations

As discussed during the lectures, the transport of fluid, heat and solutes in the subsurface are all diffusion processes. The equations of these processes are very similar, as you can see:

\begin{equation}
    S_s \dfrac{\partial h}{\partial t} = \nabla K_f \nabla h + W_f
\end{equation}

\begin{equation}
    \rho_b c_b \dfrac{\partial T}{\partial t} = \nabla K_h \nabla T + W_h
\end{equation}

\begin{equation}
    \phi \dfrac{\partial C}{\partial t} = \nabla K_s \nabla C + W_s
\end{equation}

subscripts f, h and s denote the diffusion coefficient and source term for fluid, heat and solute.

In the remaining part of this document we will use a generalized diffusion equation to explain how your numerical model of transient fluid, heat or solute transport. The general equation is:

\begin{equation}
    S\dfrac{\partial u}{\partial t} = \nabla K \nabla u + W
\end{equation}

For one-dimensional fluid flow the storage term is equal to the specific storage (S_s). The specific storage is the change in fluid pressure and hydraulic head as a result of a change in fluid volume, and is a function of the compressibility of the rock matrix and water (see your lecture notes). Values of specific storage range from 10^-4^ to 10^-6^. The specific yield is the volume of water released from unconfined aquifers due to a change in watertable, ie. simply the volume of water that is drained from the porous rock or sediments. This ranges from 0.05 for clays to 0.4 for coarse sands.

For the model runs in this exercise you will have to change the coefficients *S*, *u*, *K* and *W* to the right coefficients for your case study. We will discuss how to setup the script for your case study further on, but lets first go through the theory of how transient fluid flow can be modeled the finite difference approximation. 

## Solving the transient diffusion equation

In exercise 1 and 2 we used Euler's method to solve the steady-state groundwater flow equation. Euler's method for solving partial differential equations is to discretize derivatives: ie. to quantify derivatives at a discrete interval. Mathematically the method looks like this:

\begin{equation}
    \dfrac{\partial u}{\partial x} = \dfrac{u(x+\Delta x) - u(x)}{\Delta x}
\end{equation}

where *u(x+$\Delta$ x)* denotes the value of *u* at position $x + \Delta x$.

We will again use Euler's method to solve the partial differential equation of transient diffusion. First we simplify the diffusion equation to a 1D depth integrated equation by replacing $\nabla q$ by $\partial  q/ \partial x$:

\begin{equation}
    S\dfrac{\partial u}{\partial t} = - \dfrac{\partial q}{\partial x}  + W
\end{equation}

where q is a flux of fluid, heat or solutes. In this equation we have 2 derivatives, one derivative of the variable *u* over time, $\dfrac{\partial u}{\partial t}$, and one derivative of the flux (*q*) over space ($\dfrac{\partial q}{\partial x}$).

Now lets discretize these following Euler's method. First, lets get rid of the spatial derivative of the flux (*q*):

\begin{equation}
    S\dfrac{\partial u_i}{\partial t} = - \dfrac{q_{i+1/2} - q_{i-1/2}}{2 \Delta x} + W
\end{equation}

The subscript ~i+1/2~ means the flux (*q*) from node *i* to the next node over *i+1*. Similarly subscript ~i-1/2~ means the flux (*q*) from the node before the current one (*i-1*) to the current node (*i*).

Lets also replace the derivative of u over time with the discretized version:

\begin{equation}
    S\dfrac{u_{i,j+1} - u_{i,j}}{\Delta t} = - \dfrac{q_{i+1/2,j} - q_{i-1/2,j}}{2 \Delta x} + W
\end{equation}

In this equation the index term *i* denotes the position in space and *j* denotes the timestep. *i* would be the current node, *i-1* one node to the left, *i+1* one node to the right. Similarly, *j* is the current timestep, *j+1* is the next timestep.

We can rearrange this equation to yield an equation for the value of variable *u* at timestep *j+1*:

\begin{equation}
    u_{i,j+1} = - \dfrac{\Delta t}{S} \dfrac{q_{i+1/2,j} - q_{i-1/2,j}}{2 \Delta x} + \dfrac{\Delta t W}{S} + u_{i,j}
\end{equation}

This equation simply states that the value of variable *u* at node position *i* and timestep *j+1* is a function of of the flux towards this node at timestep *j*, a source term *W* and the value of the variable at the previous timestep *j*.

To complete the equation we need to calculate the flux towards node *i*. We can do this again by applying Euler's method. The equation for a diffusive flux of fluid, heat or solutes is:

\begin{equation}
    q = - K \dfrac{\partial u}{\partial x}
\end{equation}

in discretized form this is

\begin{equation}
    q_{1/2} = - K \dfrac{(u_{0} - u_1)}{\Delta x}
\end{equation}
 
Note that the notation for flux from node 0 to node 1 is $q_{1/2}$.

Note also that if you use depth-integrated fluid flux then you have to multiply hydraulic conductivity (*K*) by the thickness (*b*) in this equation.

## Transient diffusion in Python

Now we have two equations to solve the transient diffusion equation, one equation that calculates the flux *q* at timestep *j* and one that calculates the value of variable *u* at timestep *j+1* using the flux at timestep *j* and the value of *u* at timestep *j*.

The two equations are implemented in a python script (surprisingly called exercise_3.py). We will use this script and adjust the variables to model your fluid flow problem of choice.

The new part of the script where the transient diffusion equation is solved looks like this:

~~~~python
for j in range(1, n_timesteps):
    
    # calculate the flux between nodes
    q = -K * (h[j-1, 1:] - h[j-1, :-1]) / dx

    # set specified variable value at the left-hand node:
    h[j, 0] = h0[j]

    # implement no-flow boundary condition at right-hand side:
    q_bottom = 0.0
    h[j, -1] = h[j-1, -1] + (dt/S) * (-(q_bottom - q[-1])/dx) + (dt / S) * W[-1]

    # update nodes in the middle:
    h[j, 1:-1] = h[j-1, 1:-1] + (dt/S)*(-(q[1:] - q[:-1])/dx) + (dt/S) * W[1:-1]
~~~~

This part contains a loop through all timesteps (``for j in range(1, n_timesteps):``). Any line that is indented (shifted to the right) below this for loop is considered part of the loop by Python. 

Next Python calculates the fluid flux between each node in your numerical model ``q = -K * (h[j-1, 1:] - h[j-1, :-1]) / dx``. Hopefully by now you recognise that this is Darcy's law of fluid flow, where fluid flow is a function of the hydraulic gradient (``(h[j-1, 1:] - h[j-1, :-1]) / dx``) and the hydraulic conductivity (``K``). The index ``[j-1, 1:]`` means that Python looks at the value of h at the last timestep (``j-1``) and then looks at all the nodes except the first one (``1:``).

Next there are three lines that calculate the hydraulic head at the left, middle and right-hand nodes at the current timestep j as a function of the flux (q), the size of the timestep, storativity and the value of hydraulic head at the last timestep (``h[j-1]``). For the top node the hydraulic head is fixed ``h[j, 0] = h0[j]``. For the lowest node ``h[j, -1]`` the flux from the bottom is 0. This ensures that there is no fluid flow over the bottom boundary of our system. For the middle nodes the hydraulic head at the new timestep is simply a function of how much water flows towards or away from it ((q[1:] - q[:-1])/dx), along with the hydraulic head at the previous timestep, storativity and timestep size.

Next: Choose a case study for the exercise. There are two choices:

1. Diffusion of fluid pressure and seismicity below Mt. Hood

2. Climate resilience of a qanat system in a semi-arid environment

Choose one of these exercises and move over to the jupyter notebook of one of these two case studies.